In [ ]:
# IMPORT PACKAGES 
from PIL import Image, ImageDraw
import numpy as np
from os import path, makedirs
import time

%matplotlib inline
import matplotlib.path as pltpath
from matplotlib.pyplot import imshow

In [ ]:
# SIMULATION HEAD 
from PIL import Image, ImageDraw
from tqdm import tqdm

def sim(J,Z,params,boundary):
    for t in tqdm(range(0,params['time'])):
        # determine myosin motor state
        # myosinX motor state forward 0->1->2
        # myosinX motor state backward 2->1->0  
        tempJ = J
        # apply forward change of states on myosin motors
        for j in range(0,params['motors']):
            if tempJ[0,j] != 2:
                tempJ[0,j],tempJ[1,j],tempJ[3,j],tempJ[4,j] = motor_state_forward(j,tempJ,params,boundary);
                if tempJ[0,j] == 2 and tempJ[0,tempJ[1,j].astype(int)] != 2:
                    tempJ[0,tempJ[1,j].astype(int)] = 2
                    tempJ[1,tempJ[1,j].astype(int)] = j
            
            # move newly paired myosin motor bundles to the same location, halfway between original locations
            if tempJ[0,j] == 2 and J[0,j] != 2 and tempJ[2,j] == -1 and tempJ[2,tempJ[1,j].astype(int)] == -1 and \
               j<tempJ[1,j]:
                    tempJ[3,j] = (tempJ[3,j] + tempJ[3,tempJ[1,j].astype(int)])/2
                    tempJ[4,j] = (tempJ[4,j] + tempJ[4,tempJ[1,j].astype(int)])/2
                    tempJ[3,tempJ[1,j].astype(int)] = tempJ[3,j]
                    tempJ[4,tempJ[1,j].astype(int)] = tempJ[4,j]
        
        # apply backward change of states on myosin motors
        for j in range(0,params['motors']):
            if tempJ[0,j] == 1 and params['inactivationRate']*params['tstep']>np.random.uniform():
                tempJ[0,j] = 0
            elif tempJ[0,j] == 2 and tempJ[2,j] == -1 and tempJ[2,tempJ[1,j].astype(int)] == -1 and \
            params['unbundleRate']*params['tstep']>np.random.uniform():
                tempJ[0,tempJ[1,j].astype(int)] = 1
                tempJ[1,tempJ[1,j].astype(int)] = -1
                tempJ[0,j] = 1
                tempJ[1,j] = -1
                    
        J = tempJ
        J[:3,:]
                
            
        # check if myosin motors attach or detach from filaments
        # call sub-routine if myosinX motor is bundled and has not been edited yet
        cands = np.where(J[1,:]>J[1,J[1,:].astype(int)])[0]
        if len(cands>0):
            for j in cands:
                J[2,j],J[2,J[1,j].astype(int)],J[3,j],J[4,j],J[3,J[1,j].astype(int)],J[4,J[1,j].astype(int)] = \
                motor_attach_detach(J[2,j].astype(int),J[2,J[1,j].astype(int)].astype(int),
                                     J[3,j],J[4,j],
                                     J[3,J[1,j].astype(int)],J[4,J[1,j].astype(int)],
                                     Z,params,boundary)

                
        # determine movement of myosin motors that are bound to filaments
        cands = np.where( ((J[2,:]!=-1) | (J[2,J[1,:].astype(int)]!=-1)) & (J[1,:]>J[1,J[1,:].astype(int)]) )[0]
        for j in cands:
            # call sub-routine if myosin motor is bundled, attached to a filament and has not been edited yet
            J[2,j],J[2,J[1,j].astype(int)],J[3,j],J[4,j],J[3,J[1,j].astype(int)],J[4,J[1,j].astype(int)] = \
            bound_motor_movement(J[2,j].astype(int),J[2,J[1,j].astype(int)].astype(int),
                                  J[3,j],J[4,j],
                                  J[3,J[1,j].astype(int)],J[4,J[1,j].astype(int)],
                                  Z,params)
        
        
        # check if any filaments undergo depolymerization
        for f in range(0,params['filaments']):
            # check if filament depolymerizes
            if params['p4']*params['tstep']>np.random.uniform():
                J,Z[0,f],Z[1,f],Z[2,f],Z[3,f],Z[4,f] = depolymerization(J,f,params,boundary)
        
        
        # iteratively determine filament movement
        for f in range(0,params['filaments']):
            # determine force and movement on filament
            Z[0,f],Z[1,f],Z[2,f],old_xp,old_yp = force_on_filament(J,Z[0,f],Z[1,f],Z[2,f],Z[3,f],Z[4,f],f,params)
            Z[3,f] = Z[0,f] + params['fil_length']*np.cos(Z[2,f])
            Z[4,f] = Z[1,f] + params['fil_length']*np.sin(Z[2,f])
            # check if filament hit boundaries
            Z[:,f] = filament_boundary_check(Z,f,params,boundary)
            
            # move motors attached to filament accordingly
            # find motors on filament
            mots1 = np.where((J[0,:]==2) & (J[0,J[1,:].astype(int)]==2) & (J[2,:]!=J[2,J[1,:].astype(int)]) & (J[2,:]!=-1) & (J[2,J[1,:].astype(int)]!=-1) & (J[2,:]==f))[0]
            # distance between motor and plus end of filament
            s = np.sqrt( (old_xp - J[3,mots1])**2 + (old_yp - J[4,mots1])**2 );
            # x and y movement of motor to be on filament
            J[3,mots1] = Z[0,f] + s*np.cos(Z[2,f]);
            J[4,mots1] = Z[1,f] + s*np.sin(Z[2,f]);

            # find motors with bundled motor on filament
            mots2 = np.where((J[0,:]==2) & (J[0,J[1,:].astype(int)]==2) & (J[2,:]!=J[2,J[1,:].astype(int)]) & (J[2,:]!=-1) & (J[2,J[1,:].astype(int)]!=-1) & (J[2,J[1,:].astype(int)]==f))[0];
            # distance between motor and plus end of filament
            s = np.sqrt((old_xp - J[3,J[1,mots2].astype(int)])**2 + (old_yp - J[4,J[1,mots2].astype(int)])**2);
            # x and y movement of motor to be on filament
            J[3,J[1,mots2].astype(int)] = Z[0,f] + s*np.cos(Z[2,f]);
            J[4,J[1,mots2].astype(int)] = Z[1,f] + s*np.sin(Z[2,f]);  
            
        
        # check if any bundled myosin motors have surpassed stretch radius limit
        # every motor on a filament (group1)
        v1 = np.where(J[2,:]!=-1)[0]
        # bundle index for each myoX motor in v1 (group2)
        v2 = J[1,v1].astype(int)
        # bound filament index for v1 myo motors (group3)
        v3 = J[2,v1]
        # bound filament index for v2 myo motors (group4)
        v4 = J[2,v2]
        # myoX motors of group4 that are not bound to a filament
        r1 = np.where(v4==-1)[0]
        # myoX motor bundles where both legs are bound to same filament
        r2 = np.where(v3==v4)[0]
        # group1 with exclusions (r1 and r2)
        g1 = np.delete(v1,np.union1d(r1,r2))
        # find stretch distances between candidate myosin motors
        motor_stretch = np.sqrt((J[3,g1] - J[3,J[1,g1].astype(int)])**2 + (J[4,g1] - J[4,J[1,g1].astype(int)])**2)
        # find indices of stretch that surpass bundled myosin stretch limit
        overstretched = np.where(motor_stretch>params['r'])[0]
        # for bundled pairs passing radius, randomly select a leg to unbind from filament and move to other leg's position
        for motor in overstretched:
            mot = g1[motor]
            if np.random.uniform()<0.5:
                J[2,mot] = -1
                J[3,mot] = J[3,J[1,mot].astype(int)]
                J[4,mot] = J[4,J[1,mot].astype(int)]
            else:
                J[2,J[1,mot].astype(int)] = -1
                J[3,J[1,mot].astype(int)] = J[3,mot]
                J[4,J[1,mot].astype(int)] = J[4,mot]

    
        # write image data to '.tiff' file
        if params['save_image']==1:
            if (t+params['image_iter']+1)%params['image_iter'] == 0:
                time_point = str(int(((t+1)/params['image_iter'])))

                order = len(str(params['time']))
                lead_zeros = order - len(str(time_point))
                built_name = ''
                for i in range(lead_zeros):
                    built_name+='0'
                name = params['image_save_path']+params['image_save_name']+built_name+str(time_point)+'.png'
                
                im_png = Image.fromarray(image_build(J,Z,params))
                im_png.save(name)
                
    # finish and pass data back
    return J,Z

In [ ]:
# SUB-ROUTINES 
def motor_boundary_check(x,y,params,boundary):
    if ~boundary.contains_point(np.array([x,y])):
        h = params['boundary_height'];
        w = params['boundary_width'];
        b = [np.less(y,-1*h),np.greater(y,h),np.less(x,-1*w),np.greater(x,w)];
        # b[0] = y position below bottom boundary
        # b[1] = y position above top boundary
        # b[2] = x position left of left boundary
        # b[3] = x position right of right boundary
        if b[0] == True and b[2] == False and b[3] == False:
            y = -1*h;
        elif b[1] == True and b[2] == False and b[3] == False:
            y = h;
        elif b[2] == True and b[0] == False and b[1] == False:
            x = -1*w;
        elif b[3] == True and b[0] == False and b[1] == False:
            x = w;
        elif b[0] == True and b[2] == True:
            x = -1*w;
            y = -1*h;
        elif b[0] == True and b[3] == True:
            x = w;
            y = -1*h;
        elif b[1] == True and b[2] == True:
            x = -1*w;
            y = h;
        elif b[1] == True and b[3] == True:
            x = w;
            y = h; 
            
    return x,y


def motor_state_forward(j,J,params,boundary):
    # check if motor ungergoes activation event
    if J[0,j] == 0:
        # define state transition
        if params['activationRate']*params['tstep'] > np.random.uniform():
            J[0,j] = 1
        # define motor diffusion movement
        J[3,j] = J[3,j] + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn()
        J[4,j] = J[4,j] + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn()
        J[3,j],J[4,j] = motor_boundary_check(J[3,j],J[4,j],params,boundary)
        
    
    # check if motor undergoes bundling event
    elif J[0,j] == 1:
        # condition 1: look for motor candidates to bundle with
        candidate1 = np.where(J[0,:] == 1)[0]
        
        # condition 2: look for motors within motorBundleRadius
        candidate2 = np.where(np.sqrt((J[3,candidate1]-J[3,j])**2 + \
                              (J[4,candidate1]-J[4,j])**2)<params['motorBundleRadius'])[0]

        # group the candidate lists together (returns indices of candidates)
        bundle_candidates = [candidate1[i] for i in candidate2]

        # check the probability of a bundling event
        if (len(bundle_candidates)*params['tstep']*params['bundleRate']>np.random.uniform() 
            and len(bundle_candidates)>1):
            # uniformly randomly select a motor to bundle with                              
            J[1,j] = bundle_candidates[int(np.ceil(len(bundle_candidates)*np.random.uniform()))-1]
            while J[1,j]==j:
                J[1,j] = bundle_candidates[int(np.ceil(len(bundle_candidates)*np.random.uniform()))-1]
            J[0,j] = 2
        else:
            # diffusion of motor occurs if no bundling event
            J[3,j] = J[3,j] + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn()
            J[4,j] = J[4,j] + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn()
            J[3,j],J[4,j] = motor_boundary_check(J[3,j],J[4,j],params,boundary)
        
    return J[0,j],J[1,j],J[3,j],J[4,j]


def find_filaments_2D(Z,x,y,params):
    # test cadidates
    d0 = x - Z[0,:];
    d1 = Z[3,:] - Z[0,:];
    d2 = y - Z[1,:];
    d3 = Z[4,:] - Z[1,:];

    d = (d0*d1 + d2*d3) / \
        (d1**2 + d3**2)
    T1 = (d>0) & (d<1)

    var = ((d0*d3)-(d2*d1))**2 / \
          (d1**2+d3**2)
    dpp = d0**2 + d2**2
    dmm = (x-Z[3,:])**2 + (y-Z[4,:])**2

    T4 = np.where((var < params['r']**2) & (T1 == True) | \
         ((dpp < params['r']**2) ^ (dmm < params['r']**2)) & (T1 == False))[0]
    
    return len(T4),T4


def update_motor_position(x1,y1,x2,y2,xm,ym):
    # find closest position on line to motor as a fraction of the filament length
    var = ((xm-x1)*(x2-x1) + (ym-y1)*(y2-y1)) / ((x2-x1)**2 + (y2-y1)**2);
    if var>1:
        var = 1;
    elif var<0:
        var = 0;
    # calculate new x and y positions for motor
    new_xm = x1 + var*(x2-x1);
    new_ym = y1 + var*(y2-y1);
    
    return new_xm,new_ym


def motor_attach_detach(lfil,rfil,lx,ly,rx,ry,Z,params,boundary):
    # toggle switches; disallows right motor from performing event if switched (left motor event occured)
    fil_switch = 0;    
    diffuse_event = 0;
    detach_event = 0;
    
    # check left motor events
    if lfil<0: # left motor not bound to filament
        # find number of filament binding cadidates and their indices for left motor
        l_num_fils,l_fil_indices = find_filaments_2D(Z,lx,ly,params);  
        if l_num_fils*params['tstep']*params['p1']>np.random.uniform():
            # select a random filament for left motor to bind to
            lfil = l_fil_indices[int(np.ceil(l_num_fils*np.random.uniform()))-1];
            # set new position of newly attached left motor
            lx,ly = update_motor_position(Z[0,lfil],Z[1,lfil],Z[3,lfil],Z[4,lfil],lx,ly);
            # switch fil_switch toggle
            fil_switch = 1;
        else:
            diffuse_event = 1;
            lx = rx;
            ly = ry;
    else:
        # check if both motors are attached to the same filament
        # to see if left motor switches binding to a different filament
        if lfil == rfil and np.random.uniform()>0.5:
            fil_switch = 1;
            # find number of filament binding cadidates and their indices for left motor
            l_num_fils,l_fil_indices = find_filaments_2D(Z,lx,ly,params);
            # probability of binding event
            if l_num_fils*params['tstep']*params['p1']>np.random.uniform():
                # select a random filament for left motor to bind to
                lfil = l_fil_indices[int(np.ceil(l_num_fils*np.random.uniform()))-1];
                # set new position of newly attached left motor
                lx,ly = update_motor_position(Z[0,lfil],Z[1,lfil],Z[3,lfil],Z[4,lfil],lx,ly);
        # check for detachment/unbinding event of left motor
        if np.random.uniform()>0.5:   
            detach_event = 1;
            # probability of left motor detachment from filament
            if params['p0']*params['tstep']>np.random.uniform():
                lfil = -1;
                lx = rx;
                ly = ry;
    
    # check right motor events
    if rfil<0: # right motor not bound to filament
        rx = lx;
        ry = ly;
        # find number of filament binding cadidates and their indices for right motor
        r_num_fils,r_fil_indices = find_filaments_2D(Z,rx,ry,params);
        # probability of binding event
        if r_num_fils*params['tstep']*params['p1']>np.random.uniform():
            # select a random filament for right motor to bind to
            rfil = r_fil_indices[int(np.ceil(r_num_fils*np.random.uniform()))-1];
            # set new position of newly attached right motor
            rx,ry = update_motor_position(Z[0,rfil],Z[1,rfil],Z[3,rfil],Z[4,rfil],rx,ry);
            if lfil<0:
                lx = rx;
                ly = ry;
        # check if both motors in bundle unbound to a filament; diffusion event occurs
        elif diffuse_event == 1:
            lx = lx + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn();
            ly = ly + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn();
            # check if new positions are within boundary
            lx,ly = motor_boundary_check(lx,ly,params,boundary);
            rx = lx;
            ry = ly;
    else: # right motor bound to filament
        # check if right motor and left motor are bound to same filament
        # and if left motor did not have a filament switch event
        if rfil==lfil and fil_switch == 0:
            # find number of filament binding cadidates and their indices for right motor
            r_num_fils,r_fil_indices = find_filaments_2D(Z,rx,ry,params);
            # probability of binding event
            if r_num_fils*params['tstep']*params['p1']>np.random.uniform():
                # select a random filament for right motor to bind to
                rfil = r_fil_indices[int(np.ceil(r_num_fils*np.random.uniform()))-1];
                # set new position of newly attached right motor
                rx,ry = update_motor_position(Z[0,rfil],Z[1,rfil],Z[3,rfil],Z[4,rfil],rx,ry);
        # check if both detachment event for left motor and filament switch of 
        # either motor did not occur
        if detach_event == 0 and fil_switch == 0:    
            # probability of right motor detachment from filament
            if params['p0']*params['tstep']>np.random.uniform():
                rfil = -1;
                rx = lx;
                ry = ly;
        
    return lfil,rfil,lx,ly,rx,ry


def bound_motor_movement(lfil,rfil,lx,ly,rx,ry,Z,params):
    # left motor not bound, right motor bound to filament; right motor walks
    if lfil==-1 and rfil!=-1:
        cb = np.sqrt((Z[3,rfil] - rx)**2 + (Z[4,rfil] - ry)**2);
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            rx = Z[0,rfil];
            ry = Z[1,rfil];
            rfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,rfil] - rx)**2 + (Z[1,rfil] - ry)**2) - (params['v']*params['tstep']);
            rx = Z[0,rfil] + (s*np.cos(Z[2,rfil]));
            ry = Z[1,rfil] + (s*np.sin(Z[2,rfil]));
        # update location of bundled motor
        lx = rx;
        ly = ry;
        
    # right motor not bound, left motor bound to filament; left motor walks    
    elif lfil!=-1 and rfil==-1:
        cb = np.sqrt((Z[3,lfil] - lx)**2 + (Z[4,lfil] - ly)**2);
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            lx = Z[0,lfil];
            ly = Z[1,lfil];
            lfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,lfil] - lx)**2 + (Z[1,lfil] - ly)**2) - (params['v']*params['tstep']);
            lx = Z[0,lfil] + (s*np.cos(Z[2,lfil]));
            ly = Z[1,lfil] + (s*np.sin(Z[2,lfil]));
        # update location of bundled motor
        rx = lx;
        ry = ly;
        
    # both motors bound to separate filaments
    elif lfil!=-1 and rfil!=-1 and lfil!=rfil:
        # calculate travel distance for left motor
        cb = np.sqrt((Z[3,lfil] - lx)**2 + (Z[4,lfil] - ly)**2);
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            lx = Z[0,lfil];
            ly = Z[1,lfil];
            lfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,lfil] - lx)**2 + (Z[1,lfil] - ly)**2) - (params['v']*params['tstep']);
            lx = Z[0,lfil] + (s*np.cos(Z[2,lfil]));
            ly = Z[1,lfil] + (s*np.sin(Z[2,lfil]));
        # calculate travel distance for right motor
        cb = np.sqrt((Z[3,rfil] - rx)**2 + (Z[4,rfil] - ry)**2);
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            rx = Z[0,rfil];
            ry = Z[1,rfil];
            rfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,rfil] - rx)**2 + (Z[1,rfil] - ry)**2) - (params['v']*params['tstep']);
            rx = Z[0,rfil] + (s*np.cos(Z[2,rfil]));
            ry = Z[1,rfil] + (s*np.sin(Z[2,rfil]));      
        # calculate new distance between bundled motors
        motor_stretch = np.sqrt((lx-rx)**2 + (ly-ry)**2);
        # check if bundled motors have passed stretch threshold
        # if so, both motors "fall off" and unbind
        if motor_stretch>params['r']:
            lfil = -1;
            rfil = -1;
            # equal chance of one motor moving to position of other motor
            if np.random.uniform()>0.5:
                lx = rx;
                ly = ry;
            else:
                rx = lx;
                ry = ly;
                
    # both motors bound to the same filament    
    elif lfil==rfil and lfil!=-1:
        # calculate travel distance for left motor
        cb = np.sqrt((Z[3,lfil] - lx)**2 + (Z[4,lfil] - ly)**2);    
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            lx = Z[0,lfil];
            ly = Z[1,lfil];
            lfil = -1;
            rfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,lfil] - lx)**2 + (Z[1,lfil] - ly)**2) - (params['v']*params['tstep']);
            lx = Z[0,lfil] + (s*np.cos(Z[2,lfil]));
            ly = Z[1,lfil] + (s*np.sin(Z[2,lfil]));
        # update location of bundled motor
        rx = lx;
        ry = ly;

    return lfil,rfil,lx,ly,rx,ry        


def depolymerization(J,fil,params,boundary):
    # remove all bound motors from filament
    for j in range(0,params['motors']):
        # check if motor is active and bundled
        if J[0,j] == 2:
            # if motor j is on filament, but motor bundled to j is not
            if J[2,j]==fil and J[2,J[1,j].astype(int)]!=fil:
                # mark motor j as unbound and move it to position of bundled motor
                J[2,j] = -1;
                J[3,j] = J[3,J[1,j].astype(int)];
                J[4,j] = J[4,J[1,j].astype(int)];
            # if motor bundled to j is on filament, but motor j is not
            elif J[2,j]!=fil and J[2,J[1,j].astype(int)]==fil:
                # mark bundled motor as unbound and move it to position of motor j
                J[2,J[1,j].astype(int)] = -1;
                J[3,J[1,j].astype(int)] = J[3,j];
                J[4,J[1,j].astype(int)] = J[4,j];
            # if both bundled motors are on filament
            elif J[2,j]==fil and J[2,J[1,j].astype(int)]==fil:
                # mark both bundled motors as unbound
                J[2,j] = -1;
                J[2,J[1,j].astype(int)] = -1;
                # equal probability of one motor to move to position of the other
                if np.random.uniform()>0.5:
                    J[3,j] = J[3,J[1,j].astype(int)];
                    J[4,j] = J[4,J[1,j].astype(int)];
                else:
                    J[3,J[1,j].astype(int)] = J[3,j];
                    J[4,J[1,j].astype(int)] = J[4,j];
    # reset position of filament (to keep number of filaments constant
    # by assuming for each filament deploymerized, one is polymerized)
    x_plus_end = 2*params['boundary_width']*np.random.uniform()-params['boundary_width'];
    y_plus_end = 2*params['boundary_height']*np.random.uniform()-params['boundary_height'];
    while 1:
        angle = 2*np.pi*np.random.uniform();
        x_minus_end = x_plus_end+params['fil_length']*np.cos(angle);
        y_minus_end = y_plus_end+params['fil_length']*np.sin(angle);
        # check if minus ends of filament are within boundary
        if boundary.contains_point(np.array([x_minus_end,y_minus_end])):
            break 
    fil_params = [x_plus_end,y_plus_end,angle,x_minus_end,y_minus_end];

    return J,x_plus_end,y_plus_end,angle,x_minus_end,y_minus_end

        

    

    
def filament_boundary_check(Z,f,params,boundary):
    xp = Z[0,f]; 
    yp = Z[1,f];
    angle = Z[2,f];
    xm = Z[3,f];
    ym = Z[4,f];
    
    h = params['boundary_height'];
    w = params['boundary_width'];
    # check if plus end of filament is within boundary
    if ~boundary.contains_point(np.array([xp,yp])):
        b = [np.less(yp,-1*h),np.greater(yp,h),np.less(xp,-1*w),np.greater(xp,w)];
        # b[0] = yp position below bottom boundary
        # b[1] = yp position above top boundary
        # b[2] = xp position left of left boundary
        # b[3] = xp position right of right boundary
        if b[0] == True and b[2] == False and b[3] == False:
            yp = -1*h;
            ym = yp + params['fil_length']*np.sin(angle);
        elif b[1] == True and b[2] == False and b[3] == False:
            yp = h;
            ym = yp + params['fil_length']*np.sin(angle);
        elif b[2] == True and b[0] == False and b[1] == False:
            xp = -1*w;
            xm = xp + params['fil_length']*np.cos(angle);
        elif b[3] == True and b[0] == False and b[1] == False:
            xp = w;
            xm = xp + params['fil_length']*np.cos(angle);
        elif b[0] == True and b[2] == True:
            xp = -1*w;
            yp = -1*h;
            xm = xp + params['fil_length']*np.cos(angle);
            ym = yp + params['fil_length']*np.sin(angle);
        elif b[0] == True and b[3] == True:
            xp = w;
            yp = -1*h;
            xm = xp + params['fil_length']*np.cos(angle);
            ym = yp + params['fil_length']*np.sin(angle);
        elif b[1] == True and b[2] == True:
            xp = -1*w;
            yp = h;
            xm = xp + params['fil_length']*np.cos(angle);
            ym = yp + params['fil_length']*np.sin(angle);
        elif b[1] == True and b[3] == True:
            xp = w;
            yp = h; 
            xm = xp + params['fil_length']*np.cos(angle);
            ym = yp + params['fil_length']*np.sin(angle);
    # check if minus end of filament is within boundary
    if ~boundary.contains_point(np.array([xm,ym])):
        b = [np.less(ym,-1*h),np.greater(ym,h),np.less(xm,-1*w),np.greater(xm,w)];
        # b[0] = ym position below bottom boundary
        # b[1] = ym position above top boundary
        # b[2] = xm position left of left boundary
        # b[3] = xm position right of right boundary
        if b[0] == True and b[2] == False and b[3] == False:
            ym = -1*h;
            yp = ym - params['fil_length']*np.sin(angle);
        elif b[1] == True and b[2] == False and b[3] == False:
            ym = h;
            yp = ym - params['fil_length']*np.sin(angle);
        elif b[2] == True and b[0] == False and b[1] == False:
            xm = -1*w;
            xp = xm - params['fil_length']*np.cos(angle);
        elif b[3] == True and b[0] == False and b[1] == False:
            xm = w;
            xp = xm - params['fil_length']*np.cos(angle);
        elif b[0] == True and b[2] == True:
            xm = -1*w;
            ym = -1*h;
            xp = xm - params['fil_length']*np.cos(angle);
            yp = ym - params['fil_length']*np.sin(angle);
        elif b[0] == True and b[3] == True:
            xm = w;
            ym = -1*h;
            xp = xm - params['fil_length']*np.cos(angle);
            yp = ym - params['fil_length']*np.sin(angle);
        elif b[1] == True and b[2] == True:
            xm = -1*w;
            ym = h;
            xp = xm - params['fil_length']*np.cos(angle);
            yp = ym - params['fil_length']*np.sin(angle);
        elif b[1] == True and b[3] == True:
            xm = w;
            ym = h; 
            xp = xm - params['fil_length']*np.cos(angle);
            yp = ym - params['fil_length']*np.sin(angle);

    return [xp,yp,angle,xm,ym]


def force_on_filament(J,xp,yp,angle,xm,ym,fil,params):
    # frictional coefficients
    eta = params['viscosity']
    di = 0.008; # diameter of actin filament in um
    length = params['fil_length'];
    p = length/di;
    gparl = -0.2;
    gperp = 0.84;
    grot = -0.662;  
    l_pr = (2*np.pi*eta*length)/(np.log(p) + gparl);
    l_pp = (4*np.pi*eta*length)/(np.log(p) + gperp);
    l_r  = ((1/3)*np.pi*eta*(length**3))/(np.log(p) + grot);
    # thermal diffusion of filament paramters
    KT = 0.005;
    Dpr = KT/l_pr;
    Dpp = KT/l_pp;
    Dr  = KT/l_r;

    # pre_allocate force vector component arrays and torque arrays
    force_vectors = np.zeros((2,params['motors']));
    torques = np.zeros(params['motors']);
    
    # rotation matrix and inverse
    rot = np.array([[np.cos(angle),np.sin(angle)],[-1*np.sin(angle),np.cos(angle)]]);
    rot_inv = np.array([[np.cos(angle),-1*np.sin(angle)],[np.sin(angle),np.cos(angle)]]);
    
    # center of mass position of filament
    COMx = (xp + xm)/2;
    COMy = (yp + ym)/2;
    COM = np.array([[COMx],[COMy]]);
    # center of mass position in rotated coordinate system; "matmul = matrix multiplication"
    COM_rot = np.matmul(rot,COM);

    # Calculate forces from myosin
    # every motor on the filament (group1)
    v1 = np.where(J[2,:]==fil)[0];
    # bundle index for each motor in v1 (group2)
    v2 = J[1,v1].astype(int)
    # bound filament index for v2 motors (group3)
    v3 = J[2,v2];
    # motors of group4 that are not bound to the filament
    r1 = np.where(v3==-1)[0];
    # motor bundles where both legs are bound to same filament
    r2 = np.where(fil==v3)[0];
    # group1 with exclusions (r1 and r2)
    g1 = np.delete(v1,np.union1d(r1,r2));
    # position of first motor
    x1 = J[3,g1];
    y1 = J[4,g1];
    # position of second motor
    x2 = J[3,J[1,g1].astype(int)];
    y2 = J[4,J[1,g1].astype(int)];
    # compute x and y components of force vector
    F_x = params['k']*(x2 - x1);
    F_y = params['k']*(y2 - y1);
    # compute length of motor from center of mass of filament
    lever = np.sqrt((x1 - COMx)**2 + (y1 - COMy)**2);
    # check which side of COM the motor is
    for mot in range(0,len(lever)):
        if np.sqrt((xp - x1[mot])**2 + (yp - y1[mot])**2)<length/2:
            lever[mot] = -1*lever[mot];
    # rotate x and y components into parallel and perpendicular components
    # then insert parallel and perpindicular components of force vector into array
    force_vectors[0,g1] = rot[0,0]*F_x + rot[0,1]*F_y;
    force_vectors[1,g1] = rot[1,0]*F_x + rot[1,1]*F_y;
    torques[g1] = lever*force_vectors[1,g1];
    
    # pre-allocate array for new COM positions in rotated coordinate system      
    COM_rot_new = np.zeros([2,1]);    

    # new parallel component of COM of filament
    COM_rot_new[0] = COM_rot[0] + \
                     params['tstep']*np.sum(force_vectors[0,:])/l_pr + \
                     np.sqrt(2*params['tstep']*Dpr)*np.random.randn()
    # new perpendicular component of COM of filament
    COM_rot_new[1] = COM_rot[1] + \
                     params['tstep']*np.sum(force_vectors[1,:])/l_pp + \
                     np.sqrt(2*params['tstep']*Dpp)*np.random.randn()
    # rotate COM positions in rotated cooridnate system back to standard coordinate system
    COM_new = np.matmul(rot_inv,COM_rot_new);
    # new angular component of filament
    angle_new = angle + \
                params['tstep']*np.sum(torques)/l_r + \
                np.sqrt(2*params['tstep']*Dr)*np.random.randn()
    # update positions of plus end of filament
    xp_new = COM_new[0] - (length/2)*np.cos(angle_new)
    yp_new = COM_new[1] - (length/2)*np.sin(angle_new)
    
    return xp_new,yp_new,angle_new,xp,yp


def image_build(J,Z,params):
    r = params['res']
    s = tuple(params['s'])
    h = params['boundary_height']
    w = params['boundary_width']
    
    # configure data for image construction
    Zpx = (Z[0,:]+(w))*((s[0]/w/2)*r)
    Zpy = (Z[1,:]+(h))*((s[1]/h/2)*r)
    Zmx = (Z[3,:]+(w))*((s[0]/w/2)*r)
    Zmy = (Z[4,:]+(h))*((s[1]/h/2)*r)
    
    Jx1 = (J[3,:]+(w))*((s[0]/w/2)*r)
    Jy1 = (J[4,:]+(h))*((s[1]/h/2)*r)
    Jx2 = (J[3,J[1,:].astype(int)]+(w))*((s[0]/w/2)*r)
    Jy2 = (J[4,J[1,:].astype(int)]+(h))*((s[1]/h/2)*r)

    # create new PIL image object
    im = Image.new('RGB', s)
    draw = ImageDraw.Draw(im)

    # plot filament positions
    [draw.line(((Zpx[i],Zpy[i]),(Zmx[i],Zmy[i])),fill=(255,0,0)) for i in range(params['filaments'])] # red
    
    
    # find all bundled pairs of myosin motors
    motorsX = (np.where(J[1,:]!=-1)[0]) 
    # plot motor bundles
    [draw.line(((Jx1[i],Jy1[i]),(Jx2[i],Jy2[i])),fill=(0,255,0)) for i in motorsX] # green
    # plot myosin motor positions
    [draw.point([(Jx1[i],Jy1[i])],fill=(255,255,255)) for i in range(params['motors'])] # white

    return np.asarray(im);


def initialize_positions_box(params,boundary,J,Z):
    boundary_height = params['boundary_height'];
    boundary_width = params['boundary_width'];
    boundary = pltpath.Path([(-1*boundary_width,-1*boundary_height),
                          (-1*boundary_width,boundary_height),
                          (boundary_width,boundary_height),
                          (boundary_width,-1*boundary_height)],
                          readonly=True)
    
    # assign random positions to ends of filaments
    for z in range(0,params['filaments']):
        # randomly assign positions to plus ends of filaments
        Z[0,z] = 2*(boundary_width/params['init_zone_factor'])*np.random.uniform()-boundary_width;
        Z[1,z] = 2*boundary_height*np.random.uniform()-boundary_height;
        while 1:
            angle = 2*np.pi*np.random.uniform();
            x_minus_end = Z[0,z]+params['fil_length']*np.cos(angle);
            y_minus_end = Z[1,z]+params['fil_length']*np.sin(angle);
            # check if minus ends of filament are within boundary
            if boundary.contains_point(np.array([x_minus_end,y_minus_end])):
                Z[2,z] = angle;
                Z[3,z] = x_minus_end;
                Z[4,z] = y_minus_end;
                break      
          
    # assign random positions to myosin II motors
    for j in range(0,params['motors']):
        J[3,j] = 2*(boundary_width/params['init_zone_factor'])*np.random.uniform()-boundary_width;
        J[4,j] = 2*boundary_height*np.random.uniform()-boundary_height;
                
    return J,Z

In [ ]:
# INITIALIZATION - DEFINE PARAMETERS HERE 
# define dictionary to store all parameters
params = {
# image parameters
'save_image':1, #0=No, 1=Yes
'res':1, #set resolution (r=1 == s[0] x s[1] pixels)
's':(500,500), # size of image in pixels
'image_iter':1,
'image_save_path':"./sim_ims/",
'image_save_name':"sim_",

# simulation parameters
'time':500, #iterations of simulation
'tstep':0.01, #time step between each iteration
'filaments':1000, #number of filaments
'motors':5000, #number of myosin II motors

# filament parameters
'fil_length':1.0, #filament length um
'p0':1.0, #myosinX detachment rate
'p1':0.2, #myosinX attachement rate
'p4':0.7, #depolymerization (turnover) rate
    
# myosin motor parameters
'r':0.3, #radius of search for motor attachment to filament & max stretch of bundle um
'v':1.0, #velocity of myosin motor um/s
'k':10.0, #stiffness of dimer 'spring' pN/um
'diffusionRate':0.0029,
'unbundleRate':1.0,
'bundleRate':0.2,
'activationRate':10.0,
'inactivationRate':1.0,
'motorBundleRadius':0.3, #same as r according to literature for myosin II um

# computational domain parameters
'boundary_width':2.0, #um
'boundary_height':2.0, #um
'viscosity':1.0, #pN*s/um^2  or  Pa*s
'init_zone_factor':1.0 #inverse of fraction of domain space on left side of domain
}

# J -> myosin II motor data:
#    J(0,:) = progression of myosin state
#        0-inactive
#        1-active
#        2-bundled & active
#    J(1,:) = index of motor bundled to (-1 means no bundle)
#    J(2,:) = index of filament bound to (-1 means no binding to filament)
#    J(3,:) = x position of motor head
#    J(4,:) = y position of motor head
# Z -> actin filament data:
#    Z(0,:) = x position of filament +end
#    Z(1,:) = y position of filament +end
#    Z(2,:) = angle of orientation (unit circle, CCW)
#    Z(3,:) = Z(1,:) - (fil_length * cos(Z(3,:))) x position of filament -end
#    Z(4,:) = Z(2,:) - (fil_length * sin(Z(3,:))) y position of filament -end


from PIL import Image, ImageDraw
from os import path, makedirs

# check if save path exists; if not, create
if params['save_image'] == 1:
    if not path.exists(params['image_save_path']):
        makedirs(params['image_save_path'])

# fix saved image size to match specified resolution
params['s'] = np.asarray((params['s'][0]*params['res'],params['s'][1]*params['res']))

# pre-allocate memory for output arrays
J = np.zeros((5,params['motors']))
J[[1,2],:] = -1
Z = np.zeros((5,params['filaments']))

# define boundary
boundary_height = params['boundary_height'];
boundary_width = params['boundary_width'];
boundary = pltpath.Path([(-1*boundary_width,-1*boundary_height),
                      (-1*boundary_width,boundary_height),
                      (boundary_width,boundary_height),
                      (boundary_width,-1*boundary_height)],
                      readonly=True)

# initialize positions
J,Z = initialize_positions_box(params,boundary,J,Z)

# write image data to file
if params['save_image']==1:
    order = len(str(params['time']))
    lead_zeros = order - 1
    built_name = ''
    for i in range(lead_zeros):
        built_name+='0'
    name = params['image_save_path']+params['image_save_name']+built_name+'0'+'.png'

    im_png = Image.fromarray(image_build(J,Z,params))
    im_png.save(name)
    imshow(im_png)
    

In [ ]:
# RUN THE SIMULATION 

J,Z = sim(J,Z,params,boundary)


# import cProfile
# cProfile.run('sim(J,Z,params,boundary)')

In [ ]:
# VIEW IMAGES 
import glob
from IPython.html.widgets import interact
from IPython.display import Image, display

cache = {}
def get_image(key):
    if not key in cache:
        cache[key] = open(key,'rb').read()
    return cache[key]

images = glob.glob(params['image_save_path']+'*.png')
for image in images:
    get_image(image)

def pltimg(Time=0):
    order = len(str(params['time']))
    lead_zeros = order - len(str(Time))
    built_name = ''
    for i in range(lead_zeros):
        built_name+='0'
    filename = params['image_save_path']+'sim_'+built_name+str(Time)+'.png'
    display(Image(data=cache[filename]))

interact(pltimg, Time=(0,params['time'],params['image_iter']))

In [ ]:
# CONVERT '.PNG' IMAGES INTO A '.TIFF' STACK 
tiff_name = 'sim_k=110_eta=1.tif'
path = './'

from scipy import misc
from tqdm import tqdm
from tifffile import imsave

first = 0
for image_path in tqdm(glob.glob("./sim_ims/"+'*.png')):
    im = misc.imread(image_path)
    if first == 0:
        imsave(path+tiff_name,im,compress=1)
        first = 1
    else:
        imsave(path+tiff_name,im,append=True,compress=1)